In [ ]:
#TODO since dual logic implementation with optimization is a pain, and we are already marking such candles as loss, might as well skip it for now and come back to it later.
# def calculate_wins_losses_optimized_with_dual_logic(master_dictionary, win_perc=0.73, loss_perc=0.4):
#     for symbol in master_dictionary["symbols"]:
#         for chart_time in master_dictionary["chart_times"]:
#             try:
#                 print(f"Calculating for {symbol} {chart_time}")

#                 # Define the directory for processed data
#                 processed_data_dir = Path(output_dir) / f"{symbol}-{chart_time}/processed_data"

#                 # Create the directory if it doesn't exist
#                 processed_data_dir.mkdir(parents=True, exist_ok=True)

#                 # Construct the file names
#                 og_file_name = f"{symbol}-{chart_time}.csv"
#                 og_file_path = Path(output_dir) / f"{symbol}-{chart_time}/{og_file_name}"
#                 new_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"
#                 new_file_path = processed_data_dir / new_file_name

#                 # Read the CSV file into a dataframe
#                 df = pd.read_csv(og_file_path)

#                 # Initialize new columns
#                 df["if_short"] = np.nan
#                 df["if_long"] = np.nan
#                 df["long_target"] = df["entry"] * (1 + win_perc / 100)
#                 df["short_target"] = df["entry"] * (1 - win_perc / 100)
#                 df["long_stop_loss"] = df["entry"] * (1 - loss_perc / 100)
#                 df["short_stop_loss"] = df["entry"] * (1 + loss_perc / 100)
#                 df["shorts_win_after"] = np.nan
#                 df["longs_win_after"] = np.nan
#                 df["shorts_lose_after"] = np.nan
#                 df["longs_lose_after"] = np.nan
#                 df["dual_loss"] = 0
#                 df["entered_before"] = np.nan

#                 # Define functions to compute win/loss results
#                 def compute_win_loss(row):
#                     entry_idx = row.name
#                     long_target_reached = df.loc[entry_idx:, 'high'].ge(row['long_target']).idxmax() if df.loc[entry_idx:, 'high'].ge(row['long_target']).any() else np.nan
#                     long_stop_loss_reached = df.loc[entry_idx:, 'low'].le(row['long_stop_loss']).idxmax() if df.loc[entry_idx:, 'low'].le(row['long_stop_loss']).any() else np.nan
#                     short_target_reached = df.loc[entry_idx:, 'low'].le(row['short_target']).idxmax() if df.loc[entry_idx:, 'low'].le(row['short_target']).any() else np.nan
#                     short_stop_loss_reached = df.loc[entry_idx:, 'high'].ge(row['short_stop_loss']).idxmax() if df.loc[entry_idx:, 'high'].ge(row['short_stop_loss']).any() else np.nan

#                     if long_target_reached is not np.nan and (long_stop_loss_reached is np.nan or long_target_reached < long_stop_loss_reached):
#                         df.at[entry_idx, 'if_long'] = 1
#                         df.at[entry_idx, 'longs_win_after'] = long_target_reached - entry_idx
#                     elif long_stop_loss_reached is not np.nan:
#                         df.at[entry_idx, 'if_long'] = -1
# #                         df.at[entry_idx, 'longs_lose_after'] = long_stop_loss_reached - entry_idx
# #                         df.at[entry_idx, 'dual_loss'] = 1
# #                         df.at[entry_idx, 'entered_before'] = long_stop_loss_reached - entry_idx

#                     if short_target_reached is not np.nan and (short_stop_loss_reached is np.nan or short_target_reached < short_stop_loss_reached):
#                         df.at[entry_idx, 'if_short'] = 1
#                         df.at[entry_idx, 'shorts_win_after'] = short_target_reached - entry_idx
#                     elif short_stop_loss_reached is not np.nan:
#                         df.at[entry_idx, 'if_short'] = -1
# #                         df.at[entry_idx, 'shorts_lose_after'] = short_stop_loss_reached - entry_idx
# #                         df.at[entry_idx, 'dual_loss'] = 1
# #                         df.at[entry_idx, 'entered_before'] = short_stop_loss_reached - entry_idx

#                 # Apply the function to each row
#                 tqdm.pandas(desc=f"Processing {symbol}-{chart_time}")
#                 df.apply(compute_win_loss, axis=1)

#                 # Save the processed data
#                 df.to_csv(new_file_path, index=False)
#                 print(f"Processed file saved as {new_file_name}")

#             except Exception as e:
#                 print(f"Error processing {symbol} {chart_time}: {e}")



In [ ]:
def calculate_wins_losses_old(master_dictionary, win_perc=0.73, loss_perc=0.4):
    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            try:
                print(f"Calculating for {symbol} {chart_time}")

                # Define the directory for processed data
                processed_data_dir = Path(output_dir) / f"{symbol}-{chart_time}/processed_data"

                # Create the directory if it doesn't exist
                if not processed_data_dir.exists():
                    processed_data_dir.mkdir(parents=True, exist_ok=True)

                # Construct the file name
                og_file_name = f"{symbol}-{chart_time}.csv"
                og_file_path = Path(output_dir) / f"{symbol}-{chart_time}/{og_file_name}"
                new_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"
                new_file_path = processed_data_dir / new_file_name

                # Read the CSV file into a dataframe
                df = pd.read_csv(og_file_path)

                # Initialize new columns
                df["if_short"] = 0
                df["if_long"] = 0
                df["long_target"] = np.nan
                df["short_target"] = np.nan
                df["long_stop_loss"] = np.nan
                df["short_stop_loss"] = np.nan
                df["shorts_win_after"] = np.nan
                df["longs_win_after"] = np.nan
#                 df["dual_loss"] = 0
#                 df["entered_before"] = np.nan


                # Calculate targets and stop losses, then determine wins and losses
                for i in tqdm(range(len(df)), desc=f"Processing {symbol}-{chart_time}", unit='row'):
                    if pd.notna(df.loc[i, 'entry']):
                        long_target = df.loc[i, 'entry'] * (1 + win_perc / 100)
                        long_stop_loss = df.loc[i, 'entry'] * (1 - loss_perc / 100)
                        short_target = df.loc[i, 'entry'] * (1 - win_perc / 100)
                        short_stop_loss = df.loc[i, 'entry'] * (1 + loss_perc / 100)

                        # Initialize columns for current row
                        df.loc[i, 'if_long'] = np.nan
                        df.loc[i, 'longs_win_after'] = np.nan
                        df.loc[i, 'if_short'] = np.nan
                        df.loc[i, 'shorts_win_after'] = np.nan
                        df.loc[i, 'dual_loss'] = 0
                        df.loc[i, 'entered_before'] = np.nan
                        df.loc[i, 'long_target'] = long_target
                        df.loc[i, 'long_stop_loss'] = long_stop_loss
                        df.loc[i, 'short_stop_loss'] = short_stop_loss
                        

                        # Evaluate long trades
                        for j in range(i, len(df)):
                            if df.loc[j, 'high'] >= long_target:
                                if df.loc[j, 'low'] <= long_stop_loss:
                                    df.loc[i, 'if_long'] = -1
#                                     df.loc[i, 'dual_loss'] = 1
#                                     df.loc[i, 'entered_before'] = j - i
                                else:
                                    df.loc[i, 'if_long'] = 1
                                    df.loc[i, 'longs_win_after'] = j - i
                                break
                            elif df.loc[j, 'low'] <= long_stop_loss:
                                df.loc[i, 'if_long'] = -1
                                break
                        df.loc[i, 'short_target'] = short_target
                        # Evaluate short trades
                        for j in range(i, len(df)):
                            if df.loc[j, 'low'] <= short_target:
                                if df.loc[j, 'high'] >= short_stop_loss:
                                    df.loc[i, 'if_short'] = -1
#                                     df.loc[i, 'dual_loss'] = 1
#                                     df.loc[i, 'entered_before'] = j - i
                                else:
                                    df.loc[i, 'if_short'] = 1
                                    df.loc[i, 'shorts_win_after'] = j - i
                                break
                            elif df.loc[j, 'high'] >= short_stop_loss:
                                df.loc[i, 'if_short'] = -1
                                break

                # Save the processed data
                df.to_csv(new_file_path, index=False)
                print(f"Processed file saved as {new_file_name}")

            except Exception as e:
                print(f"Error processing {symbol} {chart_time}: {e}")



In [ ]:
def calculate_wins_losses_optimized(master_dictionary, win_perc=0.73, loss_perc=0.4, lookahead_window=10000):
    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            try:
                print(f"Calculating for {symbol} {chart_time}")

                # Define the directory for processed data
                processed_data_dir = Path(output_dir) / f"{symbol}-{chart_time}/processed_data"

                # Create the directory if it doesn't exist
                processed_data_dir.mkdir(parents=True, exist_ok=True)

                # Construct the file names
                og_file_name = f"{symbol}-{chart_time}.csv"
                og_file_path = Path(output_dir) / f"{symbol}-{chart_time}/{og_file_name}"
                new_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"
                new_file_path = processed_data_dir / new_file_name

                # Read the CSV file into a dataframe
                df = pd.read_csv(og_file_path)

                # Initialize new columns
                df["if_short"] = np.nan
                df["if_long"] = np.nan
                df["long_target"] = df["entry"] * (1 + win_perc / 100)
                df["short_target"] = df["entry"] * (1 - win_perc / 100)
                df["long_stop_loss"] = df["entry"] * (1 - loss_perc / 100)
                df["short_stop_loss"] = df["entry"] * (1 + loss_perc / 100)
                df["shorts_win_after"] = np.nan
                df["longs_win_after"] = np.nan

                # Convert DataFrame columns to numpy arrays for faster processing
                highs = df['high'].values
                lows = df['low'].values
                long_targets = df['long_target'].values
                short_targets = df['short_target'].values
                long_stop_losses = df['long_stop_loss'].values
                short_stop_losses = df['short_stop_loss'].values

                # Define functions to compute win/loss results
                def compute_win_loss(row):
                    entry_idx = row.name
                    # Define the lookahead window range
                    lookahead_end = min(entry_idx + lookahead_window, len(df))
                    
                    # Slice the future highs and lows from current index onwards, respecting the lookahead window
                    future_highs = highs[entry_idx:lookahead_end]
                    future_lows = lows[entry_idx:lookahead_end]

                    long_target_reached = np.argmax(future_highs >= row['long_target']) if np.any(future_highs >= row['long_target']) else np.nan
                    long_stop_loss_reached = np.argmax(future_lows <= row['long_stop_loss']) if np.any(future_lows <= row['long_stop_loss']) else np.nan
                    short_target_reached = np.argmax(future_lows <= row['short_target']) if np.any(future_lows <= row['short_target']) else np.nan
                    short_stop_loss_reached = np.argmax(future_highs >= row['short_stop_loss']) if np.any(future_highs >= row['short_stop_loss']) else np.nan

                    # Long trade logic
                    if not np.isnan(long_target_reached) and (np.isnan(long_stop_loss_reached) or long_target_reached < long_stop_loss_reached):
                        df.at[entry_idx, 'if_long'] = 1
                        df.at[entry_idx, 'longs_win_after'] = long_target_reached
                    elif not np.isnan(long_stop_loss_reached):
                        df.at[entry_idx, 'if_long'] = -1

                    # Short trade logic
                    if not np.isnan(short_target_reached) and (np.isnan(short_stop_loss_reached) or short_target_reached < short_stop_loss_reached):
                        df.at[entry_idx, 'if_short'] = 1
                        df.at[entry_idx, 'shorts_win_after'] = short_target_reached
                    elif not np.isnan(short_stop_loss_reached):
                        df.at[entry_idx, 'if_short'] = -1

                # Apply the function to each row with a progress bar
                print(f"Processing dataframe for {symbol}-{chart_time}...")
                tqdm.pandas(desc=f"Processing Rows")
                df.progress_apply(compute_win_loss, axis=1)

                # Save the processed data
                df.to_csv(new_file_path, index=False)
                print(f"Processed file saved as {new_file_name}")

            except Exception as e:
                print(f"Error processing {symbol} {chart_time}: {e}")

In [ ]:
def calculate_wins_losses_optimized_big_timeframes(master_dictionary, win_perc=0.73, loss_perc=0.4):
    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            try:
                print(f"Calculating for {symbol} {chart_time}")

                # Define the directory for processed data
                processed_data_dir = Path(output_dir) / f"{symbol}-{chart_time}/processed_data"
                processed_data_dir.mkdir(parents=True, exist_ok=True)

                # Construct the file names
                og_file_name = f"{symbol}-{chart_time}.csv"
                og_file_path = Path(output_dir) / f"{symbol}-{chart_time}/{og_file_name}"
                new_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"
                new_file_path = processed_data_dir / new_file_name

                # Read the CSV file into a dataframe
                df = pd.read_csv(og_file_path)

                # Initialize new columns
                df["if_short"] = np.nan
                df["if_long"] = np.nan
                df["long_target"] = df["entry"] * (1 + win_perc / 100)
                df["short_target"] = df["entry"] * (1 - win_perc / 100)
                df["long_stop_loss"] = df["entry"] * (1 - loss_perc / 100)
                df["short_stop_loss"] = df["entry"] * (1 + loss_perc / 100)
                df["shorts_win_after"] = np.nan
                df["longs_win_after"] = np.nan

                # Compute targets and stop losses
                high = df['high'].values
                low = df['low'].values
                long_targets = df['long_target'].values
                long_stop_losses = df['long_stop_loss'].values
                short_targets = df['short_target'].values
                short_stop_losses = df['short_stop_loss'].values

                # Use tqdm for progress tracking
                print(f"Processing dataframe for {symbol}-{chart_time}...")
                for entry_idx in tqdm(range(len(df)), desc="Processing Rows", unit='row'):
                    long_target_reached = np.nan
                    long_stop_loss_reached = np.nan
                    short_target_reached = np.nan
                    short_stop_loss_reached = np.nan

                    if not np.isnan(long_targets[entry_idx]):
                        future_high = high[entry_idx:]
                        if np.any(future_high >= long_targets[entry_idx]):
                            long_target_reached = np.where(future_high >= long_targets[entry_idx])[0][0] + entry_idx
                        if np.any(low[entry_idx:] <= long_stop_losses[entry_idx]):
                            long_stop_loss_reached = np.where(low[entry_idx:] <= long_stop_losses[entry_idx])[0][0] + entry_idx

                    if not np.isnan(short_targets[entry_idx]):
                        future_low = low[entry_idx:]
                        if np.any(future_low <= short_targets[entry_idx]):
                            short_target_reached = np.where(future_low <= short_targets[entry_idx])[0][0] + entry_idx
                        if np.any(high[entry_idx:] >= short_stop_losses[entry_idx]):
                            short_stop_loss_reached = np.where(high[entry_idx:] >= short_stop_losses[entry_idx])[0][0] + entry_idx

                    # Update DataFrame based on computed values
                    if long_target_reached is not np.nan and (np.isnan(long_stop_loss_reached) or long_target_reached < long_stop_loss_reached):
                        df.at[entry_idx, 'if_long'] = 1
                        df.at[entry_idx, 'longs_win_after'] = long_target_reached - entry_idx
                    elif long_stop_loss_reached is not np.nan:
                        df.at[entry_idx, 'if_long'] = -1

                    if short_target_reached is not np.nan and (np.isnan(short_stop_loss_reached) or short_target_reached < short_stop_loss_reached):
                        df.at[entry_idx, 'if_short'] = 1
                        df.at[entry_idx, 'shorts_win_after'] = short_target_reached - entry_idx
                    elif short_stop_loss_reached is not np.nan:
                        df.at[entry_idx, 'if_short'] = -1

                # Save the processed data
                df.to_csv(new_file_path, index=False)
                print(f"Processed file saved as {new_file_name}")

            except Exception as e:
                print(f"Error processing {symbol} {chart_time}: {e}")


In [ ]:
def calculate_wins_losses_optimized_yotpaaaaa(master_dictionary, win_perc=0.73, loss_perc=0.4):
    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            try:
                print(f"Calculating for {symbol} {chart_time}")

                # Define the directory for processed data
                processed_data_dir = Path(output_dir) / f"{symbol}-{chart_time}/processed_data"
                processed_data_dir.mkdir(parents=True, exist_ok=True)

                # Construct the file names
                og_file_name = f"{symbol}-{chart_time}.csv"
                og_file_path = Path(output_dir) / f"{symbol}-{chart_time}/{og_file_name}"
                new_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"
                new_file_path = processed_data_dir / new_file_name

                # Read the CSV file into a dataframe
                df = pd.read_csv(og_file_path)

                # Initialize new columns
                df["if_short"] = np.nan
                df["if_long"] = np.nan
                df["long_target"] = df["entry"] * (1 + win_perc / 100)
                df["short_target"] = df["entry"] * (1 - win_perc / 100)
                df["long_stop_loss"] = df["entry"] * (1 - loss_perc / 100)
                df["short_stop_loss"] = df["entry"] * (1 + loss_perc / 100)
                df["shorts_win_after"] = np.nan
                df["longs_win_after"] = np.nan

                # Convert columns to numpy arrays for faster indexing
                high = df['high'].values
                low = df['low'].values
                long_targets = df['long_target'].values
                long_stop_losses = df['long_stop_loss'].values
                short_targets = df['short_target'].values
                short_stop_losses = df['short_stop_loss'].values

                # Efficiently process each row
                for entry_idx in tqdm(range(len(df)), desc="Processing Rows", unit='row'):
                    long_target_reached = np.nan
                    long_stop_loss_reached = np.nan
                    short_target_reached = np.nan
                    short_stop_loss_reached = np.nan

                    if not np.isnan(long_targets[entry_idx]):
                        future_high = high[entry_idx:]
                        long_target_reached = np.argmax(future_high >= long_targets[entry_idx]) + entry_idx if np.any(future_high >= long_targets[entry_idx]) else np.nan
                        long_stop_loss_reached = np.argmax(low[entry_idx:] <= long_stop_losses[entry_idx]) + entry_idx if np.any(low[entry_idx:] <= long_stop_losses[entry_idx]) else np.nan

                    if not np.isnan(short_targets[entry_idx]):
                        future_low = low[entry_idx:]
                        short_target_reached = np.argmax(future_low <= short_targets[entry_idx]) + entry_idx if np.any(future_low <= short_targets[entry_idx]) else np.nan
                        short_stop_loss_reached = np.argmax(high[entry_idx:] >= short_stop_losses[entry_idx]) + entry_idx if np.any(high[entry_idx:] >= short_stop_losses[entry_idx]) else np.nan

                    # Update DataFrame based on computed values
                    if not np.isnan(long_target_reached) and (np.isnan(long_stop_loss_reached) or long_target_reached < long_stop_loss_reached):
                        df.at[entry_idx, 'if_long'] = 1
                        df.at[entry_idx, 'longs_win_after'] = long_target_reached - entry_idx
                    elif not np.isnan(long_stop_loss_reached):
                        df.at[entry_idx, 'if_long'] = -1

                    if not np.isnan(short_target_reached) and (np.isnan(short_stop_loss_reached) or short_target_reached < short_stop_loss_reached):
                        df.at[entry_idx, 'if_short'] = 1
                        df.at[entry_idx, 'shorts_win_after'] = short_target_reached - entry_idx
                    elif not np.isnan(short_stop_loss_reached):
                        df.at[entry_idx, 'if_short'] = -1

                # Save the processed data
                df.to_csv(new_file_path, index=False)
                print(f"Processed file saved as {new_file_name}")

            except Exception as e:
                print(f"Error processing {symbol} {chart_time}: {e}")


In [ ]:
def calculate_wins_losses_optimized(master_dictionary, win_perc=0.73, loss_perc=0.4):
    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            try:
                print(f"Calculating for {symbol} {chart_time}")
                
                # Define the directory for processed data
                processed_data_dir = Path(output_dir) / f"{symbol}-{chart_time}/processed_data"
                processed_data_dir.mkdir(parents=True, exist_ok=True)
                
                # Construct the file names
                og_file_path = Path(output_dir) / f"{symbol}-{chart_time}/{symbol}-{chart_time}.csv"
                new_file_path = processed_data_dir / f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"
                
                # Read the CSV file into a dataframe
                df = pd.read_csv(og_file_path)
                
                # Calculate targets and stop losses
                df["entry"] = df["open"]
                df["long_target"] = df["entry"] * (1 + win_perc / 100)
                df["short_target"] = df["entry"] * (1 - win_perc / 100)
                df["long_stop_loss"] = df["entry"] * (1 - loss_perc / 100)
                df["short_stop_loss"] = df["entry"] * (1 + loss_perc / 100)
                
                print(f"Processing dataframe for {symbol}-{chart_time}...")
                
                # Initialize result columns
                df['if_long'] = np.nan
                df['if_short'] = np.nan
                df['longs_win_after'] = np.nan
                df['shorts_win_after'] = np.nan
                
                # Compute cumulative max and min
                df['cum_max_high'] = df['high'].cummax()
                df['cum_min_low'] = df['low'].cummin()
                
                # Compute long trade outcomes
                long_target_reached = df['cum_max_high'] >= df['long_target']
                long_stop_loss_reached = df['cum_min_low'] <= df['long_stop_loss']
                df.loc[long_target_reached & ~long_stop_loss_reached, 'if_long'] = 1  # Win
                df.loc[long_stop_loss_reached, 'if_long'] = -1  # Loss
                
                # Compute short trade outcomes
                short_target_reached = df['cum_min_low'] <= df['short_target']
                short_stop_loss_reached = df['cum_max_high'] >= df['short_stop_loss']
                df.loc[short_target_reached & ~short_stop_loss_reached, 'if_short'] = 1  # Win
                df.loc[short_stop_loss_reached, 'if_short'] = -1  # Loss
                
                # Calculate win_after values for long trades
                df['longs_win_after'] = np.nan
                for i in df.index[df['if_long'] == 1]:
                    subsequent_target_hits = df.loc[i:]['cum_max_high'] >= df.loc[i, 'long_target']
                    if subsequent_target_hits.any():
                        df.loc[i, 'longs_win_after'] = subsequent_target_hits.idxmax() - i
                
                # Calculate win_after values for short trades
                df['shorts_win_after'] = np.nan
                for i in df.index[df['if_short'] == 1]:
                    subsequent_target_hits = df.loc[i:]['cum_min_low'] <= df.loc[i, 'short_target']
                    if subsequent_target_hits.any():
                        df.loc[i, 'shorts_win_after'] = subsequent_target_hits.idxmax() - i
                
                # Drop intermediate columns
                df = df.drop(['cum_max_high', 'cum_min_low'], axis=1)
                
                # Save the processed data
                df.to_csv(new_file_path, index=False)
                print(f"Processed file saved as {new_file_path.name}")
                
            except Exception as e:
                print(f"Error processing {symbol} {chart_time}: {e}")

In [ ]:
def calculate_wins_losses_batched(master_dictionary, win_perc=0.73, loss_perc=0.4, batch_size=5000):
    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            try:
                print(f"Calculating for {symbol} {chart_time}")

                # Define the directory for processed data
                processed_data_dir = Path(output_dir) / f"{symbol}-{chart_time}/processed_data"
                processed_data_dir.mkdir(parents=True, exist_ok=True)

                # Construct file paths
                og_file_name = f"{symbol}-{chart_time}.csv"
                og_file_path = Path(output_dir) / f"{symbol}-{chart_time}/{og_file_name}"
                new_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"
                new_file_path = processed_data_dir / new_file_name

                # Read the CSV file into a dataframe
                df = pd.read_csv(og_file_path)

                # Initialize new columns
                df["if_short"] = np.nan
                df["if_long"] = np.nan
                df["long_target"] = df["entry"] * (1 + win_perc / 100)
                df["short_target"] = df["entry"] * (1 - win_perc / 100)
                df["long_stop_loss"] = df["entry"] * (1 - loss_perc / 100)
                df["short_stop_loss"] = df["entry"] * (1 + loss_perc / 100)
                df["shorts_win_after"] = np.nan
                df["longs_win_after"] = np.nan

                # Convert DataFrame columns to numpy arrays for faster processing
                highs = df['high'].values
                lows = df['low'].values
                long_targets = df['long_target'].values
                short_targets = df['short_target'].values
                long_stop_losses = df['long_stop_loss'].values
                short_stop_losses = df['short_stop_loss'].values

                # Process entries in batches
                for i in tqdm(range(0, len(df), batch_size), desc=f"Processing batches for {symbol}-{chart_time}", unit='row'):
                    batch_highs = highs[i:i+batch_size]
                    batch_lows = lows[i:i+batch_size]
                    batch_size_current = len(batch_highs)

                    for j in range(batch_size_current):
                        idx = i + j  # Global index in the main dataframe
                        
                        future_highs = highs[idx:]
                        future_lows = lows[idx:]

                        # Long trade logic
                        long_hit_idx = np.argmax(future_highs >= long_targets[idx]) if np.any(future_highs >= long_targets[idx]) else np.nan
                        long_stop_idx = np.argmax(future_lows <= long_stop_losses[idx]) if np.any(future_lows <= long_stop_losses[idx]) else np.nan

                        if not np.isnan(long_hit_idx) and (np.isnan(long_stop_idx) or long_hit_idx < long_stop_idx):
                            df.at[idx, 'if_long'] = 1
                            df.at[idx, 'longs_win_after'] = long_hit_idx
                        elif not np.isnan(long_stop_idx):
                            df.at[idx, 'if_long'] = -1

                        # Short trade logic
                        short_hit_idx = np.argmax(future_lows <= short_targets[idx]) if np.any(future_lows <= short_targets[idx]) else np.nan
                        short_stop_idx = np.argmax(future_highs >= short_stop_losses[idx]) if np.any(future_highs >= short_stop_losses[idx]) else np.nan

                        if not np.isnan(short_hit_idx) and (np.isnan(short_stop_idx) or short_hit_idx < short_stop_idx):
                            df.at[idx, 'if_short'] = 1
                            df.at[idx, 'shorts_win_after'] = short_hit_idx
                        elif not np.isnan(short_stop_idx):
                            df.at[idx, 'if_short'] = -1

                # Save the processed data
                df.to_csv(new_file_path, index=False)
                print(f"Processed file saved as {new_file_name}")

            except Exception as e:
                print(f"Error processing {symbol} {chart_time}: {e}")


In [ ]:
def calculate_wins_losses_optimized_yotpa(master_dictionary, lookahead_window=10000, win_perc=0.73, loss_perc=0.4):
    for symbol in master_dictionary["symbols"]:
        for chart_time in master_dictionary["chart_times"]:
            try:
                print(f"Calculating for {symbol} {chart_time}")

                # Define the directory for processed data
                processed_data_dir = Path(output_dir) / f"{symbol}-{chart_time}/processed_data"

                # Create the directory if it doesn't exist
                processed_data_dir.mkdir(parents=True, exist_ok=True)

                # Construct the file names
                og_file_name = f"{symbol}-{chart_time}.csv"
                og_file_path = Path(output_dir) / f"{symbol}-{chart_time}/{og_file_name}"
                new_file_name = f"{symbol}-{chart_time}_W{win_perc}_L{loss_perc}.csv"
                new_file_path = processed_data_dir / new_file_name

                # Read the CSV file into a dataframe
                df = pd.read_csv(og_file_path)

                # Initialize new columns
                df["if_short"] = np.nan
                df["if_long"] = np.nan
                df["long_target"] = df["entry"] * (1 + win_perc / 100)
                df["short_target"] = df["entry"] * (1 - win_perc / 100)
                df["long_stop_loss"] = df["entry"] * (1 - loss_perc / 100)
                df["short_stop_loss"] = df["entry"] * (1 + loss_perc / 100)
                df["shorts_win_after"] = np.nan
                df["longs_win_after"] = np.nan

                # Convert DataFrame columns to numpy arrays for faster processing
                highs = df['high'].values
                lows = df['low'].values
                long_targets = df['long_target'].values
                short_targets = df['short_target'].values
                long_stop_losses = df['long_stop_loss'].values
                short_stop_losses = df['short_stop_loss'].values

                # Process each entry row by row
                for i in tqdm(range(len(df)), desc="Processing Rows", unit='row'):
                    # Slice the future highs and lows from current index onwards
                    lookahead_end = min(i + lookahead_window, len(df))
                    
                    # Slice the future highs and lows from current index onwards, respecting the lookahead window
                    future_highs = highs[i:lookahead_end]
                    future_lows = lows[i:lookahead_end]

                    # Early stopping for performance improvement
                    long_hit_idx = np.argmax(future_highs >= long_targets[i]) if np.any(future_highs >= long_targets[i]) else np.nan
                    long_stop_idx = np.argmax(future_lows <= long_stop_losses[i]) if np.any(future_lows <= long_stop_losses[i]) else np.nan

                    short_hit_idx = np.argmax(future_lows <= short_targets[i]) if np.any(future_lows <= short_targets[i]) else np.nan
                    short_stop_idx = np.argmax(future_highs >= short_stop_losses[i]) if np.any(future_highs >= short_stop_losses[i]) else np.nan

                    # Long trade logic
                    if not np.isnan(long_hit_idx) and (np.isnan(long_stop_idx) or long_hit_idx < long_stop_idx):
                        df.at[i, 'if_long'] = 1
                        df.at[i, 'longs_win_after'] = long_hit_idx
                    elif not np.isnan(long_stop_idx):
                        df.at[i, 'if_long'] = -1

                    # Short trade logic
                    if not np.isnan(short_hit_idx) and (np.isnan(short_stop_idx) or short_hit_idx < short_stop_idx):
                        df.at[i, 'if_short'] = 1
                        df.at[i, 'shorts_win_after'] = short_hit_idx
                    elif not np.isnan(short_stop_idx):
                        df.at[i, 'if_short'] = -1

                # Save the processed data
                df.to_csv(new_file_path, index=False)
                print(f"Processed file saved as {new_file_name}")

            except Exception as e:
                print(f"Error processing {symbol} {chart_time}: {e}")


In [ ]:
def calculate_indicators_using_talib(timeperiods, df):
    new_columns = pd.DataFrame()

    # List to store indicator columns
    indicator_columns = []
    indicator_columns.append(('HT_TRENDLINE', talib.HT_TRENDLINE(df['close'])))
    # indicator_columns.append(('MAMA', df['MAMA']), ('FAMA', df['FAMA']))
    # indicator_columns.append(('MAVP', df['MAVP']))
    indicator_columns.append(
        ('SAR', talib.SAR(df['high'], df['low'], acceleration=0, maximum=0)))
    indicator_columns.append(('SAREXT', talib.SAREXT(df['high'], df['low'])))
    indicator_columns.append(
        ('T3', talib.T3(df['close'], timeperiod=5, vfactor=0)))
    # Momentum Indicators
    indicator_columns.append(
        ('APO', talib.APO(df['close'], fastperiod=12, slowperiod=26)))
    indicator_columns.append(
        ('BOP', talib.BOP(df['open'], df['high'], df['low'], df['close'])))
    macd, macd_signal, macd_hist = talib.MACD(df['close'],
                                              fastperiod=12,
                                              slowperiod=26,
                                              signalperiod=9)
    indicator_columns.append(('MACD', macd))
    indicator_columns.append(('MACD_signal', macd_signal))
    indicator_columns.append(('MACD_hist', macd_hist))
    indicator_columns.append(
        ('PPO', talib.PPO(df['close'], fastperiod=12, slowperiod=26,
                          matype=0)))
    indicator_columns.append(('TRIX', talib.TRIX(df['close'])))
    indicator_columns.append(
        ('ULTOSC', talib.ULTOSC(df['high'], df['low'], df['close'])))
    indicator_columns.append(
        ('WILLR', talib.WILLR(df['high'], df['low'], df['close'])))

    #     # Not Working ATM
    #     indicator_columns.append(('STOCH', talib.STOCH(df['high'], df['low'], df['close'])))
    #     indicator_columns.append(('STOCHF', talib.STOCHF(df['high'], df['low'], df['close'])))
    #     indicator_columns.append(('STOCHRSI', talib.STOCHRSI(df['close'])))
    #     indicator_columns.append(('MACDEXT', talib.MACDEXT(df['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)))
    #     indicator_columns.append(('MACDFIX', talib.MACDFIX(df['close'], signalperiod=9)))

    #########Volume Indicators
    indicator_columns.append(
        ('AD', talib.AD(df['high'], df['low'], df['close'], df['volume'])))
    indicator_columns.append(('ADOSC',
                              talib.ADOSC(df['high'],
                                          df['low'],
                                          df['close'],
                                          df['volume'],
                                          fastperiod=3,
                                          slowperiod=10)))
    indicator_columns.append(('OBV', talib.OBV(df['close'], df['volume'])))

    #########Cycle Indicators
    indicator_columns.append(('HT_DCPERIOD', talib.HT_DCPERIOD(df['close'])))
    indicator_columns.append(('HT_DCPHASE', talib.HT_DCPHASE(df['close'])))
    phasor_inphase, phasor_quadrature = talib.HT_PHASOR(df['close'])
    indicator_columns.append(('HT_PHASOR_inphase', phasor_inphase))
    indicator_columns.append(('HT_PHASOR_quadrature', phasor_quadrature))
    # indicator_columns.append(('HT_SINE', talib.HT_SINE(df['close'])))
    indicator_columns.append(('HT_TRENDMODE', talib.HT_TRENDMODE(df['close'])))

    #########Price Transform
    indicator_columns.append(('AVGPRICE',
                              talib.AVGPRICE(df['open'], df['high'], df['low'],
                                             df['close'])))
    indicator_columns.append(
        ('MEDPRICE', talib.MEDPRICE(df['high'], df['low'])))
    indicator_columns.append(
        ('TYPPRICE', talib.TYPPRICE(df['high'], df['low'], df['close'])))
    indicator_columns.append(
        ('WCLPRICE', talib.WCLPRICE(df['high'], df['low'], df['close'])))
    #########Volatility Indicators
    indicator_columns.append(
        ('TRANGE', talib.TRANGE(df['high'], df['low'], df['close'])))
    #########Pattern Recognition
    indicator_columns.append(('CDL2CROWS',
                              talib.CDL2CROWS(df['open'], df['high'],
                                              df['low'], df['close'])))
    indicator_columns.append(('CDL3BLACKCROWS',
                              talib.CDL3BLACKCROWS(df['open'], df['high'],
                                                   df['low'], df['close'])))
    indicator_columns.append(('CDL3INSIDE',
                              talib.CDL3INSIDE(df['open'], df['high'],
                                               df['low'], df['close'])))
    indicator_columns.append(('CDL3LINESTRIKE',
                              talib.CDL3LINESTRIKE(df['open'], df['high'],
                                                   df['low'], df['close'])))
    indicator_columns.append(('CDL3OUTSIDE',
                              talib.CDL3OUTSIDE(df['open'], df['high'],
                                                df['low'], df['close'])))
    indicator_columns.append(('CDL3STARSINSOUTH',
                              talib.CDL3STARSINSOUTH(df['open'], df['high'],
                                                     df['low'], df['close'])))
    indicator_columns.append(('CDL3WHITESOLDIERS',
                              talib.CDL3WHITESOLDIERS(df['open'], df['high'],
                                                      df['low'], df['close'])))
    indicator_columns.append(('CDLABANDONEDBABY',
                              talib.CDLABANDONEDBABY(df['open'],
                                                     df['high'],
                                                     df['low'],
                                                     df['close'],
                                                     penetration=0)))

    indicator_columns.append(('CDLADVANCEBLOCK',
                              talib.CDLADVANCEBLOCK(df['open'], df['high'],
                                                    df['low'], df['close'])))
    indicator_columns.append(('CDLBELTHOLD',
                              talib.CDLBELTHOLD(df['open'], df['high'],
                                                df['low'], df['close'])))
    indicator_columns.append(('CDLBREAKAWAY',
                              talib.CDLBREAKAWAY(df['open'], df['high'],
                                                 df['low'], df['close'])))
    indicator_columns.append(
        ('CDLCLOSINGMARUBOZU',
         talib.CDLCLOSINGMARUBOZU(df['open'], df['high'], df['low'],
                                  df['close'])))
    indicator_columns.append(
        ('CDLCONCEALBABYSWALL',
         talib.CDLCONCEALBABYSWALL(df['open'], df['high'], df['low'],
                                   df['close'])))
    indicator_columns.append(('CDLCOUNTERATTACK',
                              talib.CDLCOUNTERATTACK(df['open'], df['high'],
                                                     df['low'], df['close'])))
    indicator_columns.append(('CDLDARKCLOUDCOVER',
                              talib.CDLDARKCLOUDCOVER(df['open'],
                                                      df['high'],
                                                      df['low'],
                                                      df['close'],
                                                      penetration=0)))

    indicator_columns.append(('CDLDOJI',
                              talib.CDLDOJI(df['open'], df['high'], df['low'],
                                            df['close'])))
    indicator_columns.append(('CDLDOJISTAR',
                              talib.CDLDOJISTAR(df['open'], df['high'],
                                                df['low'], df['close'])))
    indicator_columns.append(('CDLDRAGONFLYDOJI',
                              talib.CDLDRAGONFLYDOJI(df['open'], df['high'],
                                                     df['low'], df['close'])))
    indicator_columns.append(('CDLENGULFING',
                              talib.CDLENGULFING(df['open'], df['high'],
                                                 df['low'], df['close'])))
    indicator_columns.append(('CDLEVENINGDOJISTAR',
                              talib.CDLEVENINGDOJISTAR(df['open'],
                                                       df['high'],
                                                       df['low'],
                                                       df['close'],
                                                       penetration=0)))

    indicator_columns.append(('CDLEVENINGSTAR',
                              talib.CDLEVENINGSTAR(df['open'],
                                                   df['high'],
                                                   df['low'],
                                                   df['close'],
                                                   penetration=0)))
    indicator_columns.append(
        ('CDLGAPSIDESIDEWHITE',
         talib.CDLGAPSIDESIDEWHITE(df['open'], df['high'], df['low'],
                                   df['close'])))
    indicator_columns.append(('CDLGRAVESTONEDOJI',
                              talib.CDLGRAVESTONEDOJI(df['open'], df['high'],
                                                      df['low'], df['close'])))
    indicator_columns.append(('CDLHAMMER',
                              talib.CDLHAMMER(df['open'], df['high'],
                                              df['low'], df['close'])))
    indicator_columns.append(('CDLHANGINGMAN',
                              talib.CDLHANGINGMAN(df['open'], df['high'],
                                                  df['low'], df['close'])))
    indicator_columns.append(('CDLHARAMI',
                              talib.CDLHARAMI(df['open'], df['high'],
                                              df['low'], df['close'])))
    indicator_columns.append(('CDLHARAMICROSS',
                              talib.CDLHARAMICROSS(df['open'], df['high'],
                                                   df['low'], df['close'])))
    indicator_columns.append(('CDLHIGHWAVE',
                              talib.CDLHIGHWAVE(df['open'], df['high'],
                                                df['low'], df['close'])))
    indicator_columns.append(('CDLHIKKAKE',
                              talib.CDLHIKKAKE(df['open'], df['high'],
                                               df['low'], df['close'])))
    indicator_columns.append(('CDLHIKKAKEMOD',
                              talib.CDLHIKKAKEMOD(df['open'], df['high'],
                                                  df['low'], df['close'])))
    indicator_columns.append(('CDLHOMINGPIGEON',
                              talib.CDLHOMINGPIGEON(df['open'], df['high'],
                                                    df['low'], df['close'])))
    indicator_columns.append(
        ('CDLIDENTICAL3CROWS',
         talib.CDLIDENTICAL3CROWS(df['open'], df['high'], df['low'],
                                  df['close'])))
    indicator_columns.append(('CDLINNECK',
                              talib.CDLINNECK(df['open'], df['high'],
                                              df['low'], df['close'])))
    indicator_columns.append(('CDLINVERTEDHAMMER',
                              talib.CDLINVERTEDHAMMER(df['open'], df['high'],
                                                      df['low'], df['close'])))
    indicator_columns.append(('CDLKICKING',
                              talib.CDLKICKING(df['open'], df['high'],
                                               df['low'], df['close'])))
    indicator_columns.append(
        ('CDLKICKINGBYLENGTH',
         talib.CDLKICKINGBYLENGTH(df['open'], df['high'], df['low'],
                                  df['close'])))
    indicator_columns.append(('CDLLADDERBOTTOM',
                              talib.CDLLADDERBOTTOM(df['open'], df['high'],
                                                    df['low'], df['close'])))
    indicator_columns.append(('CDLLONGLEGGEDDOJI',
                              talib.CDLLONGLEGGEDDOJI(df['open'], df['high'],
                                                      df['low'], df['close'])))
    indicator_columns.append(('CDLLONGLINE',
                              talib.CDLLONGLINE(df['open'], df['high'],
                                                df['low'], df['close'])))
    indicator_columns.append(('CDLMARUBOZU',
                              talib.CDLMARUBOZU(df['open'], df['high'],
                                                df['low'], df['close'])))
    indicator_columns.append(('CDLMATCHINGLOW',
                              talib.CDLMATCHINGLOW(df['open'], df['high'],
                                                   df['low'], df['close'])))
    indicator_columns.append(('CDLMATHOLD',
                              talib.CDLMATHOLD(df['open'],
                                               df['high'],
                                               df['low'],
                                               df['close'],
                                               penetration=0)))
    indicator_columns.append(
        ('CDLMORNINGDOJISTAR',
         talib.CDLMORNINGDOJISTAR(df['open'], df['high'], df['low'],
                                  df['close'])))
    indicator_columns.append(('CDLMORNINGSTAR',
                              talib.CDLMORNINGSTAR(df['open'], df['high'],
                                                   df['low'], df['close'])))
    indicator_columns.append(('CDLONNECK',
                              talib.CDLONNECK(df['open'], df['high'],
                                              df['low'], df['close'])))
    indicator_columns.append(('CDLPIERCING',
                              talib.CDLPIERCING(df['open'], df['high'],
                                                df['low'], df['close'])))
    indicator_columns.append(('CDLRICKSHAWMAN',
                              talib.CDLRICKSHAWMAN(df['open'], df['high'],
                                                   df['low'], df['close'])))
    indicator_columns.append(
        ('CDLRISEFALL3METHODS',
         talib.CDLRISEFALL3METHODS(df['open'], df['high'], df['low'],
                                   df['close'])))
    indicator_columns.append(
        ('CDLSEPARATINGLINES',
         talib.CDLSEPARATINGLINES(df['open'], df['high'], df['low'],
                                  df['close'])))
    indicator_columns.append(('CDLSHOOTINGSTAR',
                              talib.CDLSHOOTINGSTAR(df['open'], df['high'],
                                                    df['low'], df['close'])))
    indicator_columns.append(('CDLSHORTLINE',
                              talib.CDLSHORTLINE(df['open'], df['high'],
                                                 df['low'], df['close'])))
    indicator_columns.append(('CDLSPINNINGTOP',
                              talib.CDLSPINNINGTOP(df['open'], df['high'],
                                                   df['low'], df['close'])))
    indicator_columns.append(('CDLSTALLEDPATTERN',
                              talib.CDLSTALLEDPATTERN(df['open'], df['high'],
                                                      df['low'], df['close'])))
    indicator_columns.append(('CDLSTICKSANDWICH',
                              talib.CDLSTICKSANDWICH(df['open'], df['high'],
                                                     df['low'], df['close'])))
    indicator_columns.append(('CDLTAKURI',
                              talib.CDLTAKURI(df['open'], df['high'],
                                              df['low'], df['close'])))
    indicator_columns.append(('CDLTASUKIGAP',
                              talib.CDLTASUKIGAP(df['open'], df['high'],
                                                 df['low'], df['close'])))
    indicator_columns.append(('CDLTHRUSTING',
                              talib.CDLTHRUSTING(df['open'], df['high'],
                                                 df['low'], df['close'])))
    indicator_columns.append(('CDLTRISTAR',
                              talib.CDLTRISTAR(df['open'], df['high'],
                                               df['low'], df['close'])))
    indicator_columns.append(('CDLUNIQUE3RIVER',
                              talib.CDLUNIQUE3RIVER(df['open'], df['high'],
                                                    df['low'], df['close'])))
    indicator_columns.append(
        ('CDLUPSIDEGAP2CROWS',
         talib.CDLUPSIDEGAP2CROWS(df['open'], df['high'], df['low'],
                                  df['close'])))
    indicator_columns.append(
        ('CDLXSIDEGAP3METHODS',
         talib.CDLXSIDEGAP3METHODS(df['open'], df['high'], df['low'],
                                   df['close'])))
    #########Statistic Functions
    indicator_columns.append(('LINEARREG', talib.LINEARREG(df['close'])))
    indicator_columns.append(
        ('LINEARREG_ANGLE', talib.LINEARREG_ANGLE(df['close'])))
    indicator_columns.append(
        ('LINEARREG_INTERCEPT', talib.LINEARREG_INTERCEPT(df['close'])))
    indicator_columns.append(
        ('LINEARREG_SLOPE', talib.LINEARREG_SLOPE(df['close'])))
    # new_columns['STDDEV'] = df['close'].rolling(timeperiod).std()
    indicator_columns.append(('TSF', talib.TSF(df['close'])))
    indicator_columns.append(('VAR', talib.VAR(df['close'])))
    # Iterate over the time periods
    for timeperiod in timeperiods:
        #########Overlap Studies
        indicator_columns.append((f'BB_upper_{timeperiod}',
                                  talib.BBANDS(df['close'],
                                               timeperiod=timeperiod)))
        indicator_columns.append((f'BB_middle_{timeperiod}',
                                  talib.BBANDS(df['close'],
                                               timeperiod=timeperiod)))
        indicator_columns.append((f'BB_lower_{timeperiod}',
                                  talib.BBANDS(df['close'],
                                               timeperiod=timeperiod)))
        indicator_columns.append((f'DEMA_{timeperiod}',
                                  talib.DEMA(df['close'],
                                             timeperiod=timeperiod)))
        indicator_columns.append(
            (f'EMA_{timeperiod}', talib.EMA(df['close'],
                                            timeperiod=timeperiod)))
        indicator_columns.append((f'KAMA_{timeperiod}',
                                  talib.KAMA(df['close'],
                                             timeperiod=timeperiod)))
        indicator_columns.append(
            (f'MA_{timeperiod}', talib.MA(df['close'], timeperiod=timeperiod)))
        # new_columns['MAMA'], new_columns['FAMA'] = talib.MAMA(df['close'], fastlimit=0, slowlimit=0)
        # new_columns['MAVP'] = talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0)
        indicator_columns.append((f'MIDPOINT_{timeperiod}',
                                  talib.MIDPOINT(df['close'],
                                                 timeperiod=timeperiod)))
        indicator_columns.append((f'MIDPRICE_{timeperiod}',
                                  talib.MIDPRICE(df['high'],
                                                 df['low'],
                                                 timeperiod=timeperiod)))
        indicator_columns.append(
            (f'SMA_{timeperiod}', talib.SMA(df['close'],
                                            timeperiod=timeperiod)))
        indicator_columns.append((f'TEMA_{timeperiod}',
                                  talib.TEMA(df['close'],
                                             timeperiod=timeperiod)))
        indicator_columns.append((f'TRIMA_{timeperiod}',
                                  talib.TRIMA(df['close'],
                                              timeperiod=timeperiod)))
        indicator_columns.append(
            (f'WMA_{timeperiod}', talib.WMA(df['close'],
                                            timeperiod=timeperiod)))
        indicator_columns.append((f'ADX_{timeperiod}',
                                  talib.ADX(df['high'],
                                            df['low'],
                                            df['close'],
                                            timeperiod=timeperiod)))
        indicator_columns.append((f'ADXR_{timeperiod}',
                                  talib.ADXR(df['high'],
                                             df['low'],
                                             df['close'],
                                             timeperiod=timeperiod)))
        aroon_up, aroon_down = talib.AROON(df['high'],
                                           df['low'],
                                           timeperiod=timeperiod)
        indicator_columns.append((f'AROON_up_{timeperiod}', aroon_up))
        indicator_columns.append((f'AROON_down_{timeperiod}', aroon_down))
        indicator_columns.append((f'AROONOSC_{timeperiod}',
                                  talib.AROONOSC(df['high'],
                                                 df['low'],
                                                 timeperiod=timeperiod)))
        indicator_columns.append((f'CCI_{timeperiod}',
                                  talib.CCI(df['high'],
                                            df['low'],
                                            df['close'],
                                            timeperiod=timeperiod)))
        indicator_columns.append(
            (f'CMO_{timeperiod}', talib.CMO(df['close'],
                                            timeperiod=timeperiod)))
        indicator_columns.append((f'DX_{timeperiod}',
                                  talib.DX(df['high'],
                                           df['low'],
                                           df['close'],
                                           timeperiod=timeperiod)))
        indicator_columns.append((f'MFI_{timeperiod}',
                                  talib.MFI(df['high'],
                                            df['low'],
                                            df['close'],
                                            df['volume'],
                                            timeperiod=timeperiod)))
        indicator_columns.append((f'MINUS_DI_{timeperiod}',
                                  talib.MINUS_DI(df['high'],
                                                 df['low'],
                                                 df['close'],
                                                 timeperiod=timeperiod)))
        indicator_columns.append((f'MINUS_DM_{timeperiod}',
                                  talib.MINUS_DM(df['high'],
                                                 df['low'],
                                                 timeperiod=timeperiod)))
        indicator_columns.append(
            (f'MOM_{timeperiod}', talib.MOM(df['close'],
                                            timeperiod=timeperiod)))
        indicator_columns.append((f'PLUS_DI_{timeperiod}',
                                  talib.PLUS_DI(df['high'],
                                                df['low'],
                                                df['close'],
                                                timeperiod=timeperiod)))
        indicator_columns.append((f'PLUS_DM_{timeperiod}',
                                  talib.PLUS_DM(df['high'],
                                                df['low'],
                                                timeperiod=timeperiod)))
        indicator_columns.append(
            (f'ROC_{timeperiod}', talib.ROC(df['close'],
                                            timeperiod=timeperiod)))
        indicator_columns.append((f'ROCP_{timeperiod}',
                                  talib.ROCP(df['close'],
                                             timeperiod=timeperiod)))
        indicator_columns.append((f'ROCR_{timeperiod}',
                                  talib.ROCR(df['close'],
                                             timeperiod=timeperiod)))
        indicator_columns.append((f'ROCR100_{timeperiod}',
                                  talib.ROCR100(df['close'],
                                                timeperiod=timeperiod)))
        indicator_columns.append(
            (f'RSI_{timeperiod}', talib.RSI(df['close'],
                                            timeperiod=timeperiod)))

        indicator_columns.append((f'ATR_{timeperiod}',
                                  talib.ATR(df['high'],
                                            df['low'],
                                            df['close'],
                                            timeperiod=timeperiod)))
        indicator_columns.append((f'NATR_{timeperiod}',
                                  talib.NATR(df['high'],
                                             df['low'],
                                             df['close'],
                                             timeperiod=timeperiod)))
        #########Statistic Functions
        indicator_columns.append((f'BETA_{timeperiod}',
                                  talib.BETA(df['high'],
                                             df['low'],
                                             timeperiod=timeperiod)))
        indicator_columns.append((f'CORREL_{timeperiod}',
                                  talib.CORREL(df['high'],
                                               df['low'],
                                               timeperiod=timeperiod)))
    new_columns = pd.concat([
        pd.DataFrame(data, columns=[name]) for name, data in indicator_columns
    ],
                            axis=1)
    return new_columns